# Visualizing Neuropixel Probe Locations
It can be handy to know the location and trajectory of the probes that obtain the data. Some NWB files have an **electrodes** field which store these locations. To be more precise, they contain arrays of **CCF** coordinates. [CCF](https://community.brain-map.org/t/allen-mouse-ccf-accessing-and-using-related-data-and-tools/359) is a framework which allows us to represent locations in the brain with coordinates that are relative to brain structure. This notebook uses CCF coordinate data in an extracellular electrophysiology NWB file to render the locations of the [Neuropixel](https://www.neuropixels.org/probe) probes that were used.

To be able to render locations in a brain, you don't need to know how CCF works except that it is a system of coordinates. We use **[ccf-widget](https://github.com/NeurodataWithoutBorders/ccf-widget)** as the engine by which we 3D render the brain and the probe coordinates.

### Environment Setup

In [4]:
from dandi import dandiapi
from pynwb import NWBHDF5IO
import numpy as np
from ccfwidget import CCFWidget

### Downloading an NWB File
If you don't already have a file to analyze, you can use a file from The Allen Institute's `Visual Coding - Neuropixels` dataset. If you want to choose your own file to download, set `dandiset_id` and `dandi_filepath` accordingly.

In [5]:
dandiset_id = "000021"
dandi_filepath = "sub-699733573/sub-699733573_ses-715093703.nwb"
download_loc = "../../../data/visual_coding"
filename = dandi_filepath.split("/")[-1]

In [ ]:
my_dandiset = dandiapi.DandiAPIClient().get_dandiset(dandiset_id)
file = my_dandiset.get_asset_by_path(dandi_filepath)
# this may take awhile, especially if the file to download is large
file.download(f"{download_loc}/{filename}")

print(f"Downloaded file to {download_loc}/{filename}")

### Extracting NWB CCF Coordinates
Here, you can read the NWB file you're interested in viewing. Specify your file of interest's relative file path in `nwb_filepath`. From there, the file will be read and the probe unit coordinates will be extracted and turned into a numpy array.

Note that this will only work with ecephys NWB files which have a valid **electrodes** field.

In [6]:
nwb_filepath = f"{download_loc}/{filename}"

In [7]:
### read the nwb file
io = NWBHDF5IO(nwb_filepath, mode="r", load_namespaces=True)
nwb = io.read()

Ignoring cached namespace 'hdmf-common' version 1.1.3 because version 1.5.1 is already loaded.
Ignoring cached namespace 'core' version 2.2.2 because version 2.5.0 is already loaded.


In [8]:
### read the x,y,z ccf coordinates and generate points
xs = nwb.electrodes.x
ys = nwb.electrodes.y
zs = nwb.electrodes.z
n = min(len(xs), len(ys), len(zs))
points = np.array([[xs[i], ys[i], zs[i]] for i in range(n)])

In [50]:
print(points.shape)

(2304, 3)


### Rendering
Rendering is as simple as generating the widget and displaying it. This will create embedded window with the interactive 3D rendering of your scene. 

In [51]:
ccf = CCFWidget(markers=[points])

In [ ]:
ccf